In [139]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [140]:
np.load('../0.npy')

array([ 0.50839818,  0.27644575, -0.5151608 , ...,  0.        ,
        0.        ,  0.        ])

In [141]:
#Path for exported data, numpy arrays
DATA_PATH = os.path.join('..//collected_data')

#Actions that we try to detect
actions = np.array(['hello', 'my', 'name', 'is'])

#30 videos worth of data
no_sequence = 30

#Videos are going to be 30 frames in length
sequence_length = 30

In [142]:
label_map = {label:num for num, label in enumerate(actions)}

In [143]:
label_map

{'hello': 0, 'my': 1, 'name': 2, 'is': 3}

In [144]:
sequences, label = [], []
for action in actions:
    for sequence in range(no_sequence):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        label.append(label_map[action])

In [145]:
np.array(sequences).shape

(120, 30, 1662)

In [146]:
np.array(label).shape

(120,)

In [147]:
X = np.array(sequences)
y = to_categorical(label).astype(int)

In [148]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [149]:
# Python: Launch TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [168]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [169]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [170]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
4/4 [==============================] - 3s 74ms/step - loss: 1.7253 - categorical_accuracy: 0.2895
Epoch 2/100
4/4 [==============================] - 0s 69ms/step - loss: 1.4334 - categorical_accuracy: 0.2456
Epoch 3/100
4/4 [==============================] - 0s 67ms/step - loss: 1.4119 - categorical_accuracy: 0.2982
Epoch 4/100
4/4 [==============================] - 0s 69ms/step - loss: 1.3770 - categorical_accuracy: 0.2982
Epoch 5/100
4/4 [==============================] - 0s 71ms/step - loss: 1.3667 - categorical_accuracy: 0.2368
Epoch 6/100
4/4 [==============================] - 0s 67ms/step - loss: 1.9611 - categorical_accuracy: 0.3158
Epoch 7/100
4/4 [==============================] - 0s 77ms/step - loss: 1.4067 - categorical_accuracy: 0.4649
Epoch 8/100
4/4 [==============================] - 0s 70ms/step - loss: 1.0922 - categorical_accuracy: 0.5526
Epoch 9/100
4/4 [==============================] - 0s 65ms/step - loss: 1.0151 - categorical_accuracy: 0.6316
Epoch 10/1

In [171]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_36 (LSTM)              (None, 30, 64)            442112    
                                                                 
 lstm_37 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_38 (LSTM)              (None, 64)                49408     
                                                                 
 dense_36 (Dense)            (None, 64)                4160      
                                                                 
 dense_37 (Dense)            (None, 32)                2080      
                                                                 
 dense_38 (Dense)            (None, 4)                 132       
                                                                 
Total params: 596,708
Trainable params: 596,708
Non-t

In [172]:
res = model.predict(X_test)

1/1 [==============================] - 0s 251ms/step


In [173]:
actions[np.argmax(res[2])]

'hello'

In [174]:
actions[np.argmax(y_test[2])]

'hello'

In [175]:
model.save('..//models//hand_signs.h5')

In [176]:
yhat = model.predict(X_train)

4/4 [==============================] - 0s 26ms/step


In [177]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [178]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[85,  1],
        [ 0, 28]],

       [[85,  0],
        [ 1, 28]],

       [[84,  0],
        [ 0, 30]],

       [[87,  0],
        [ 0, 27]]], dtype=int64)

In [179]:
accuracy_score(ytrue, yhat)

0.9912280701754386